<a href="https://colab.research.google.com/github/sumitasharma/MachineLearning/blob/master/convolutional_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
 !curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
 !unzip kagglecatsanddogs_3367a.zip
 

## Showing the cats and dogs from the folder. Break is put to show the very first image

In [0]:
!pip install opencv-python
import tensorflow as tf
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from tensorflow.keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='log/cats_dogs_log')

In [0]:
DATADIR = "/content/PetImages"
CATEGORIES = ["Dog", "Cat"]
for categories in CATEGORIES:
  path = os.path.join(DATADIR,categories)
  for img in os.listdir(path):
    img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
    plt.imshow(img_array, cmap='gray')
    plt.show()
    break
  break
  

## Reshaping the images to 100,100 size

In [0]:
new_array = cv2.resize(img_array, (100, 100))
plt.imshow(new_array, cmap='gray')
plt.show()

In [0]:

training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats
        print("DATADIR:" + DATADIR + "category:" + category)
        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in os.listdir(path):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (100, 100))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
               # print("Training data len" + len(training_data))
            except Exception as e:  # in the interest in keeping the output clean...
                print(str(e))
               
                
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

In [0]:
import random

random.shuffle(training_data)
print(training_data)

In [0]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, 100, 100, 1))

X = np.array(X).reshape(-1, 100, 100, 1)

In [0]:
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation


In [0]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])



In [0]:
model.save('cnn_dogs_cats_model.h5')

In [0]:
def prepare(filepath):
  img_array = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
  new_array = cv2.resize(img_array,(100,100))
  return new_array.reshape(-1,100,100,1)
model = tf.keras.models.load_model("cnn_dogs_cats_model.h5")
prediction = model.predict([prepare("/content/gdrive/My Drive/Colab Notebooks/doggie.jfif")])
print(prediction)
print(CATEGORIES[int(prediction[0][0])])
model = tf.keras.models.load_model("cnn_dogs_cats_model.h5")
prediction = model.predict([prepare("/content/gdrive/My Drive/Colab Notebooks/catie.jfif")])
print(prediction)
print(CATEGORIES[int(prediction[0][0])])


In [0]:
LOG_DIR = '/content/log/cats_dogs_log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
! curl http://localhost:6006

In [0]:
# Install
! npm install -g localtunnel

In [0]:
# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')
! cat url.txt